In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
from torchvision import datasets
from torchvision import transforms
from torchsummary import summary

CUDA_LAUNCH_BLOCKING="1"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.set_default_tensor_type('torch.cuda.FloatTensor')

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # Normalization layers
        self.dropout = nn.Dropout(p=0.5)
        
     
        self.conv1 = nn.Conv2d(3, 32, 3)
        self.pool = nn.MaxPool2d(2, 2)
        
        self.conv2 = nn.Conv2d(32, 32, 3)
        #self.pool2 = nn.MaxPool2d(2, 2)
        
        self.conv3 = nn.Conv2d(32, 64, 3)
        #self.pool3 = nn.MaxPool2d(2, 2)
        
        self.conv4 = nn.Conv2d(64, 64, 3)
        #self.pool4 = nn.MaxPool2d(2, 2)      
        
        self.fc1 = nn.Linear(64 * 12 * 12, 1)
        self.fc3 = nn.Sigmoid()

    def forward(self, x):
        x = self.pool(self.dropout(F.relu(self.conv1(x))))
        x = self.pool(self.dropout(F.relu(self.conv2(x))))
        x = self.pool(self.dropout(F.relu(self.conv3(x))))
        x = self.pool(self.dropout(F.relu(self.conv4(x))))
        x = x.view(-1, 64 * 12 * 12)
        x = self.fc1(x)
        x = self.fc3(x)
        
        return x


In [3]:

custom_model = Net().to(device)
summary(custom_model, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 222, 222]             896
           Dropout-2         [-1, 32, 222, 222]               0
         MaxPool2d-3         [-1, 32, 111, 111]               0
            Conv2d-4         [-1, 32, 109, 109]           9,248
           Dropout-5         [-1, 32, 109, 109]               0
         MaxPool2d-6           [-1, 32, 54, 54]               0
            Conv2d-7           [-1, 64, 52, 52]          18,496
           Dropout-8           [-1, 64, 52, 52]               0
         MaxPool2d-9           [-1, 64, 26, 26]               0
           Conv2d-10           [-1, 64, 24, 24]          36,928
          Dropout-11           [-1, 64, 24, 24]               0
        MaxPool2d-12           [-1, 64, 12, 12]               0
           Linear-13                    [-1, 1]           9,217
          Sigmoid-14                   

In [4]:
traindir = os.path.join('C:\\Users\\Cordu\\Desktop\\traffic sign data\\', 'train_set')
testdir = os.path.join('C:\\Users\\Cordu\\Desktop\\traffic sign data\\', 'validation_set')
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

In [5]:
train_loader = torch.utils.data.DataLoader(
    datasets.ImageFolder(traindir,
                         transforms.Compose([
                             transforms.Resize((224, 224)),
                             transforms.RandomHorizontalFlip(),
                             transforms.ToTensor(),
                             normalize,
                         ])),
    batch_size=64,
    shuffle=True,
    num_workers=0)

validation_loader = torch.utils.data.DataLoader(
    datasets.ImageFolder(testdir,
                         transforms.Compose([
                             transforms.Resize((224, 224)),
                             transforms.RandomHorizontalFlip(),
                             transforms.ToTensor(),
                             normalize,
                         ])),
    batch_size=64,
    shuffle=True,
    num_workers=0)

test_loader = torch.utils.data.DataLoader(
    datasets.ImageFolder(os.path.abspath('test'),
                         transforms.Compose([
                             transforms.Resize((224, 224)),
                             transforms.RandomHorizontalFlip(),
                             transforms.ToTensor(),
                             normalize,
                         ])),
    batch_size=64,
    shuffle=True,
    num_workers=0

    )

In [6]:
import torch.optim as optim

criterion = torch.nn.BCELoss()
optimizer = torch.optim.SGD(custom_model.parameters(), lr=0.001, momentum=0.9)
custom_model.train()

Net(
  (dropout): Dropout(p=0.5, inplace=False)
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (conv4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=9216, out_features=1, bias=True)
  (fc3): Sigmoid()
)

In [7]:
for epoch in range(5):  # loop over the dataset multiple times

    running_loss = 0.0
    running_corrects = 0.0
    for i, data in enumerate(train_loader,0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        labels = labels.unsqueeze(0).permute(1,0).to(dtype=torch.float)
        inputs = inputs.to(device)
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = custom_model(inputs).to(device)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        # print statistics
        running_loss += loss.item()
        classes = torch.round(outputs)
        running_corrects += (classes == labels).sum()

    print('[Epoch : %d, Total iterations : %5d] loss: %.3f' % (epoch + 1, i, running_loss / len(train_loader.dataset)))
    print('Accuracy {}% '.format(running_corrects * 100 / len(train_loader.dataset)))
    test_loss = 0
    correct = 0.0
    for data, target in validation_loader:
        data, target = data.to(device), target.to(device, dtype= torch.float)
        output = custom_model(data).to(device)
        target = target.unsqueeze(0).permute(1,0)
        test_loss += torch.nn.functional.binary_cross_entropy(output, target, reduction='sum').item()  # sum up batch loss
        
        pred = torch.round(output)
        correct += (pred == target).sum()

    test_loss /= len(validation_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            test_loss, correct, len(validation_loader.dataset),
            100. * correct / len(validation_loader.dataset)))

print('Finished Training')


[Epoch : 1, Total iterations :   166] loss: 0.005
Accuracy 86% 

Test set: Average loss: 0.5015, Accuracy: 3164/3911 (80%)

[Epoch : 2, Total iterations :   166] loss: 0.002
Accuracy 96% 

Test set: Average loss: 0.4396, Accuracy: 3322/3911 (84%)

[Epoch : 3, Total iterations :   166] loss: 0.001
Accuracy 98% 

Test set: Average loss: 0.2845, Accuracy: 3541/3911 (90%)

[Epoch : 4, Total iterations :   166] loss: 0.001
Accuracy 98% 

Test set: Average loss: 0.5994, Accuracy: 3236/3911 (82%)

[Epoch : 5, Total iterations :   166] loss: 0.000
Accuracy 98% 

Test set: Average loss: 0.2940, Accuracy: 3540/3911 (90%)

Finished Training


In [8]:
from PIL import Image
from matplotlib import cm

def image_loader(image_name):
    """load image, returns cuda tensor"""
    image = Image.open(image_name)
    image = loader(image).float()
    image = torch.tensor(image, requires_grad=True)
    image = image.unsqueeze(0)  #this is for VGG, may not be needed for ResNet
    return image.cuda()  #assumes that you're using GPU

In [9]:

for data, target in test_loader:
        data, target = data.to(device), target.to(device, dtype= torch.float)
        output = custom_model(data).to(device)
        target = target.unsqueeze(0).permute(1,0)
        output = torch.round(output)
        print(target)
        print(output)

tensor([[1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.]])
tensor([[1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.]], grad_fn=<RoundBackward>)
